In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
import gspread

In [5]:
from oauth2client.service_account import ServiceAccountCredentials

In [6]:
urls = pd.read_excel('URLs.xlsx')

In [7]:
#urls = urls.head(50)

In [8]:
scope = ['https://spreadsheets.google.com/feeds']

In [9]:
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)

In [10]:
client = gspread.authorize(creds)

In [11]:
spreadsheet_key = '1B9DQNoQt5YJJ4xJa6nZepPR55PJwA6tz5mNVlY4oCho'

In [12]:
book = client.open_by_key(spreadsheet_key)

In [13]:
worksheet = book.worksheet("workspace")

In [14]:
table = worksheet.get_all_values()

In [15]:
sheet_index = 1

In [16]:
first_row = ["url", "product_name", "product_price", "sale_price", "offer", "availability", "product_code"]

In [17]:
worksheet.insert_row(first_row, 1)

{'spreadsheetId': '1B9DQNoQt5YJJ4xJa6nZepPR55PJwA6tz5mNVlY4oCho',
 'updates': {'spreadsheetId': '1B9DQNoQt5YJJ4xJa6nZepPR55PJwA6tz5mNVlY4oCho',
  'updatedRange': 'workspace!A1:G1',
  'updatedRows': 1,
  'updatedColumns': 7,
  'updatedCells': 7}}

In [18]:
sheet_index = sheet_index + 1

In [19]:
for index, row in urls.iterrows():
    if not row['/']:
        continue

    if row['/'].endswith('/'):
        continue

    source = requests.get('https://www.markastok.com' + row['/'])
    content = BeautifulSoup(source.content,"lxml")

    product_name = content.find("h1",attrs={"class":"product-name"}).text.replace("\n","") if content.find("h1",attrs={"class":"product-name"}) else None
    product_price = content.find("span",attrs={"class":"currencyPrice discountedPrice"}).text.replace("\n","") if content.find("span",attrs={"class":"currencyPrice discountedPrice"}) else None
    sale_price = content.find("span",attrs={"class":"product-price"}).text if content.find("span",attrs={"class":"product-price"}) else None
    
    offer = None
    for i in content.select("div[class='detay-indirim']>span"):
        offer = i.text

    passive_child_count = 0
    if content.find("div",attrs={"class":"variantList"}):
        for i in content.find("div",attrs={"class":"variantList"}).findAll("a",attrs={"class":"passive"}):
            passive_child_count = passive_child_count + 1

    child_count = 0
    for i in content.select(".variantList>a.box-border>p"):
        child_count = child_count + 1
    
    availability = 0
    if child_count:
        availability = (child_count - passive_child_count) / child_count * 100
    
    if content.find("div",attrs={"class":"product-feature-content"}): 
        for i in content.find("div",attrs={"class":"product-feature-content"}).find_all("div"):
            i.string = ""

    product_code = content.find("div",attrs={"class":"product-feature-content"}).text.replace("\n","") if content.find("div",attrs={"class":"product-feature-content"}) else None

    new_row = [row['/'], product_name, product_price, sale_price, offer, availability, product_code]
    
    try:
        worksheet.insert_row(new_row, sheet_index)
        sheet_index = sheet_index + 1
    except Exception as e:
        print(e)
        pass
